In [15]:
from surface_dynamics.all import Origami # Setting up the imports
import numpy as np

In [16]:
def sl2_action(matrix, origami): 
    '''
        Arguments: matrix in SL(2,Z) and an origami
        Return: a resulting origami from action of matrix up to isomorphism of origami
    
    if det(matrix) != 1:
        raise ValueError
    for x in matrix:
        if x != round(x):
            raise ValueError
    '''
    
    Rinv = Matrix([[0,1],[-1,0]])
    A = Matrix([[440,0]])
    while matrix[1,0] != 0:
        if matrix[0,0] <= 0 or matrix[1,0] < 0: # Rip off an rotation
            matrix = Rinv * matrix
            A = np.insert(A, 0, np.array((0, 1)), 0) # insert an S at the beginning
        else:
            c=ceil(matrix[0,0]/matrix[1,0])
            matrix[0,0] = matrix[0,0]-c*matrix[1,0]
            matrix[0,1] = matrix[0,1]-c*matrix[1,1]
            A = np.insert(A, 0, np.array((1, c)), 0)
    if matrix[0,1] != 0:
        A = np.insert(A, 0, np.array((1, matrix[0,1])), 0)
    for a in A:
        if a[0] == 0:
            origami = origami.horizontal_symmetry().mirror()
        if a[0] == 1:
            origami = origami.horizontal_twist(a[1])
    return origami

In [17]:
# The matrices S, S^(-1), and R generate the group SL2Z
S = Matrix(ZZ, [[1,1],[0,1]])
R = Matrix(ZZ,[[0,-1],[1,0]])

# The SL2Z_Action function takes an origami ori and a 2x2 matrix A with integer coefficients as arguments
# It tests whether A is an element of SL2Z, and if so it returns the image of ori under the action of A
# If A is not an element of SL2Z, it returns an error message
def SL2Z_Action(ori, A):
    
    # Tests whether A is in SL2Z
    if det(A) != 1:
        return "Error: The input matrix is not an element of SL2Z"
        
    # List that will hold generators that produce A when multiplied in the order they appear
    # Temporary matrix that we will update as we reduce A to the identity using generators
    generators = []
    A_temp = A
    
    # Reduce A_temp to the identity by multiplying on the left by S, S^(-1), and R repeatedly
    # Simultaneously add the inverse of the matrix that is being multiplied to the end of list of generators
    while A_temp[1][0] != 0:
        if abs(A_temp[0][0])>=abs(A_temp[1][0]):
            if sgn(A_temp[0][0]) == sgn(A_temp[1][0]):
                A_temp = S^(-1)*A_temp
                generators = generators + [S]
            else:
                A_temp = S*A_temp
                generators = generators + [S^(-1)]
        else:
            A_temp = R*A_temp
            generators = generators + [R,R,R]
    if A_temp[0][0] == 1:
        while A_temp[0][1] > 0:
            A_temp = S^(-1)*A_temp
            generators = generators + [S]
        while A_temp[0][1] < 0:
            A_temp = S*A_temp
            generators = generators + [S^(-1)]
    else:
        while A_temp[0][1] < 0:
            A_temp = S^(-1)*A_temp
            generators = generators + [S]
        while A_temp[0][1] > 0:
            A_temp = S*A_temp
            generators = generators + [S^(-1)]
        A_temp = R*R*A_temp
        generators = generators + [R,R]
    
    # Test to confirm that generators in list multiply to produce A
    # If not, return error message
    product = Matrix(ZZ,[[1,0],[0,1]])
    for generator in generators:
        product = product*generator
    if product != A:
        return "Error: Decomposition into product of generators has failed"
        
    # Origami that will be returned after the action of A is applied
    A_times_ori = ori
    
    # Apply generators in list in reverse order because A is multiplied on the left of ori
    for generator in reversed(generators):
        if generator == R:
            A_times_ori = A_times_ori.horizontal_symmetry().mirror()
            # print('R')
        elif generator == S:
            A_times_ori = A_times_ori.horizontal_twist(1)
            # print('S')
        else:
            A_times_ori = A_times_ori.horizontal_twist(-1)
            # print('S_inv')
            
    return A_times_ori

In [18]:
def cycle_notation(array): 
    '''
        Argument: array of integers representing gluing instruction of origami, every element in array is glued to the next
        Return: string representing the cycle of the gluing
        Comment: assumes array is a permutation we are using which is a cycle as we are gluing the boxes
            until the edges meet, and is therefore NOT a general method for converting permutation to cycle notation, 
            it basically makes the array a string
    '''
    cycle = '('
    for i in array:
        cycle = cycle + str(int(i)) + ','
    cycle = cycle[:-1] + ')'
    return cycle

In [19]:
def enum_strata(n):
    '''
        Argument: integer n, size of origamis
        Return: array of origamis of the form two sheared rectangles stacked and aligned on left, 
            with bottom rectangle wider than top rectangle, they are in H(2)
        Comment: https://jamboard.google.com/d/1OSRaahlM2KiY1_onnQ5y6TDBOrseFGrHhtssKQc2ee0/edit?usp=sharing
            on slide 3 explains external gluing formula
    '''
    surfaces = []
    for u2 in range(2, n): # base rectangle has width [2, n-1)
        for h2 in range(1, round(n/u2) + 1): # base rectangle has height [1, n/u2)
            upperbound = min(u2, n-u2*h2 + 1) # top rectangle has width less than u2 or at most area of the top rectangle
            for u1 in range(1, upperbound): # top rectangle has width [1, upperbound)
                if (n-u2*h2)%u1 == 0: # as long as u1 divides the area, it's valid
                    h1 = (n-u2*h2)/u1
                    rectangle2 = np.arange(1, u2*h2 + 1)
                    rect2 = np.reshape(rectangle2, (h2, u2)) # base rectangle rect2
                    rect2 = np.flip(rect2,0)
                    rectangle1 = np.arange(u2*h2 + 1, n + 1)
                    rect1 = np.reshape(rectangle1, (h1, u1)) # top rectangle rect1
                    rect1 = np.flip(rect1,0)
                    hperm = '' 
                    # horizontal (right) permutation is independent of twist
                    for i in range(h1):
                        hperm += cycle_notation(rect1[i])
                    for i in range(h2):
                        hperm += cycle_notation(rect2[i])
                    for t1 in range(u1):
                        for t2 in range(u2):
                            marker = [1] * u2
                            vperm = ''
                            while sum(marker) != 0:
                                next = np.nonzero(marker)[0][0]
                                column = []
                                while marker[next] != 0:
                                    column = np.concatenate(([row[next] for row in rect2],column))
                                    marker[next] = 0
                                    if next < u1:
                                        col = [row[next] for row in rect1]
                                        column = np.concatenate((col, column))
                                        next = (u2-t2+(u1+next-t1)%u1)%u2
                                    else:
                                        next = (next+u2-t2)%u2
                                vperm += cycle_notation(np.flip(column,0))
                            origami = Origami(vperm, hperm)
                            surfaces.append(origami)
    return surfaces
    

In [20]:
print(len(enum_strata(13)))

407


In [21]:
def one_cyl_dir(n):
    '''
        Argument: integer n, size of the origamis in H(2) to be tested for one cyliner direction
        Return: array of origamis that do not have a one cylinder direction
    '''
    surfaces = enum_strata(n)
    surfaces_ncyl = []
    for origami in surfaces:
        found = false
        if origami.is_hyperelliptic:
            vgroup = origami.veech_group().coset_reps()
            for Mat in vgroup:
                Matinv = Mat^(-1)
                ori_orbit_rep = sl2_action(Matinv, origami)
                if len(ori_orbit_rep.cylinder_decomposition()) == 1:
                    found = true
                    break
        if not found:
            surfaces_ncyl.append(origami)
    return surfaces_ncyl

In [22]:
def sl2_orbit_reps(origami):
    '''
        Argument: an origami
        Return: an array of origamis that are representatives in sl2 orbits of origami
    '''
    ori_orbit_reps = []
    vgroup = origami.veech_group().coset_reps()
    for Mat in vgroup:
        Matinv = Mat^(-1)
        ori_orbit_reps.append(sl2_action(Matinv, origami))
    return ori_orbit_reps

In [23]:
def num_sl2_orbit(n):
    '''
        Argument: integer n, size of the origamis in H(2)
        Return: integer m, the number of SL(2,Z) orbits in Sn
    '''
    surfaces = enum_strata(n)
    list_orbits = []
    

In [24]:
# The count_SL_orbits functions takes as its input a list oris of origamis
# It returns the number of distinct SL2Z orbits represented by the origamis in the list
def count_SL_orbits(oris):
    orbits = []
    for ori in oris:
        # Determine if ori is represented in any of the orbits already counted
        inOrbits = False
        for orbit in orbits:
            if inOrbits:
                break
            for comparison_ori in orbit:
                if ori.is_isomorphic(comparison_ori):
                    inOrbits = True
                    break
        
        # If not, then add orbit of ori to list of orbits
        if not inOrbits:
            orbit = []
            for A in ori.veech_group().coset_reps():
                A_inv = A^(-1) # Use inverse of A because coset_reps() gives right coset reps
                A_inv_times_ori = SL2Z_Action(ori, A_inv)
                orbit.append(A_inv_times_ori)
            orbits.append(orbit)
    return orbits

In [25]:
for i in [3,5,7,11,13,17,19,23]:
    print(len(enum_strata(i)))
    print(len(count_SL_orbits(enum_strata(i))))
#    for orbit in count_SL_orbits(enum_strata(13)):
#        print(len(orbit))

2
1
17
2
55
2
240
2
407
2
940
2
1326
2
2387
2
